In [ ]:
%load_ext autoreload
%autoreload 2
import wandb as wb
import numpy as np
from dataset import TrainingDataset
from torch.utils.data import DataLoader
from torch import optim
from sklearn.model_selection import train_test_split
from models import *

In [ ]:
# change these accordingly
seed = Trainer.set_random_seed(None)
data_path = ""
target_path = ""
transform = None
dataset = TrainingDataset(data_path, target_path, transform)
train_idx, test_idx = train_test_split(np.arange(len(dataset)), test_size=0.2)
train_idx, val_idx = train_test_split(train_idx, test_size=0.2)
train_data, val_data, test_data = dataset.split(train_idx, val_idx, test_idx)

In [ ]:
config={
        "epochs": 10,
        "batch_size": 32,
        "learning_rate": 0.001,
        "model": "SimpleFNN",
        "in_features": 10,
        "hidden_features": 128,
        "hidden_layers": 5,
        "out_features": 5,
        "optimizer": "Adam",
        "loss": "CrossEntropyLoss"
    }
device = Trainer.get_target_device()

In [ ]:
wb.login()

In [ ]:
wb.init(
    # set the wandb project where this run will be logged 
    project='ml-pytorch-template',
    name="Test run",
    # track hyperparameters
    config=config
)
config = wb.config

In [ ]:
# define data loaders
train_loader = DataLoader(train_data, batch_size=config["batch_size"], shuffle=True)
val_loader = DataLoader(val_data, batch_size=config["batch_size"])
test_loader = DataLoader(test_data, batch_size=config["batch_size"])

In [ ]:
# define model, optimize and loss function
model = SimpleFNN(n_input_features=config["in_features"], n_hidden_features=config["hidden_features"], n_hidden_layers=config["hidden_layers"], n_output_features=config["out_features"]).to(device)
print(f"Number of parameters to train: {sum(p.numel() for p in model.parameters())}")
optimizer = optim.Adam(model.parameters(), lr=config["learning_rate"])
loss_fn = nn.CrossEntropyLoss()
trainer = Trainer(dataset, train_loader, val_loader, test_loader, model, optimizer, loss_fn, config["epochs"])

In [ ]:
train_loss, train_acc, validation_loss, validation_acc, test_loss, test_acc = trainer.optimizing_predictor("outs", None, False)

In [ ]:
wb.finish()